In [1]:
import os,gc
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
# from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split

from sklearn.pipeline import Pipeline
from sklearn.cross_validation import cross_val_score
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.grid_search import GridSearchCV 
from sklearn.metrics import make_scorer
# from mlens.metrics import make_scorer
# from mlens.model_selection import Evaluator

# from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import SGDClassifier
#import xgboost
# import lightgbm as lgb

from scipy.sparse import csr_matrix,hstack 
from sklearn.externals.joblib import dump,load

from copy import deepcopy


/home/luan/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/luan/.local/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
user_folder = os.path.expanduser("~")
data_folder = os.path.join(user_folder, 'E:/git/database/Toxic_Comment')
train_df = pd.read_csv(os.path.join(data_folder, 'train_preprocess.csv'), delimiter=',')
train_tfidf = load(os.path.join(data_folder, "train_features_10K.dump"))
test_df_idf = load(os.path.join(data_folder, 'test_features_10K.dump'))
gc.collect()

0

In [3]:
submission = pd.read_csv(os.path.join(data_folder, "sample_submission.csv"))

In [4]:
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']


In [5]:
seed = 20
np.random.seed(seed)

In [ ]:
roc_scorer = make_scorer(roc_auc_score, greater_is_better=True)
pars = {
    # "max_features": [50, 200],
    "n_estimators": [20,100],
    # "criterion": ["gini", "entropy"],
    "min_samples_leaf": [2, 4]
}

#ests = [('rf', RandomForestClassifier(random_state=seed))]

#evaluator = Evaluator(scorer=roc_scorer, cv=10, random_state=seed)
#evaluator.fit(train_tfidf, train_df["toxic"], ests, pars, n_iter=10)
#print("\nComparison with different parameter dists:\n\n%r" % evaluator.results)
clf = RandomForestClassifier(random_state = seed, n_estimators=50)
# roc_score = cross_val_score(clf, train_tfidf, train_df['toxic'], scoring='roc_auc')
grid = GridSearchCV(clf, pars, scoring='roc_auc', n_jobs=1, cv=5)
grid.fit(train_tfidf, train_df["toxic"],)
print("Accuracy: {0:.4f}%".format(grid.best_score_ * 100))
gc.collect()
# print(clf)
# print(roc_score)

In [11]:
print(clf)
print(roc_score)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=20, verbose=0, warm_start=False)
[0.91189561 0.91306193 0.91370311]


In [14]:
print("n_estimator is 50 \nroc_auc_score is %s" % roc_score)

n_estimator is 50 
 roc_auc_score is [0.95248742 0.9504648  0.95009881]


In [6]:
ests = {
    'rf': RandomForestClassifier(n_estimators=100, min_samples_leaf=2),
    #'lr': LogisticRegression(),
    #'svm': SGDClassifier(loss='log')
}

In [7]:
from sklearn.metrics import make_scorer
from copy import deepcopy
scorer = make_scorer(roc_auc_score, greater_is_better=True)

def predict_one(clf, X_train, y_train, X_test):
    clf = clf
    clf.fit(X_train, y_train)
    train_preds = clf.predict_proba(X_train)
    target_preds = clf.predict_proba(X_test)
    return train_preds, target_preds
    
def predict_all(clf, X_train, y_train, X_test, labels):
    train_preds_matrix = pd.DataFrame(y_train)
    test_preds_matrix = np.zeros((X_test.shape[0], len(labels)))
    test_preds_matrix = pd.DataFrame(test_preds_matrix); test_preds_matrix.columns = labels
    
    idx = 0
    for label in labels:
        print("Fit %s" % label)
        train_target = y_train[label].values
        first,second = predict_one(clf, X_train, train_target, X_test)
        
        train_preds_matrix.iloc[:,idx] = first[:,1]
        test_preds_matrix.iloc[:,idx] = second[:,1]
        idx += 1
        
        gc.collect()
        
    return train_preds_matrix, test_preds_matrix

#y_train = train_df[class_names].values
#train_preds, test_preds = predict_all(clf=LogisticRegression(), X_train=train_tfidf, y_train=train_df[class_names], X_test=test_df_idf,labels = class_names)
        


In [9]:
for key in ests.keys():
    print("Using %s Model..." % key)
    clf = ests[key]
    train_preds, test_preds = predict_all(clf=clf, X_train=train_tfidf, y_train=train_df[class_names], X_test=test_df_idf,labels = class_names)
    submission[class_names] = test_preds
    
    train_preds.to_csv(os.path.join(data_folder, '%s_train_preds.csv' % key), index=False)
    submission.to_csv(os.path.join(data_folder, '%s_submission.csv' % key), index=False)
    
    gc.collect()
    

Using rf Model...
Fit toxic
Fit severe_toxic
Fit obscene
Fit threat
Fit insult
Fit identity_hate


#### lightgbm

In [6]:
gc.collect()
train_preds = train_df[class_names]
for class_name in class_names:
    print("Fit %s" % class_name)
    train_target = train_df[class_name]
    train_sparse_matrix, valid_sparse_matrix, y_train, y_valid = train_test_split(train_tfidf, train_target, test_size=0.05, random_state=144)
    d_train = lgb.Dataset(train_sparse_matrix, label=y_train)
    d_valid = lgb.Dataset(valid_sparse_matrix, label=y_valid)
    watchlist = [d_train, d_valid]
    
    del train_target, train_sparse_matrix, valid_sparse_matrix, y_train, y_valid 
    gc.collect()
    
    params = {'learning_rate': 0.2,
          'application': 'binary',
          'num_leaves': 31,
          'verbosity': -1,
          'metric': 'auc',
          'data_random_seed': 2,
          'bagging_fraction': 0.8,
          'feature_fraction': 0.6,
          'nthread': 4,
          'lambda_l1': 1,
          'lambda_l2': 1}
    rounds_lookup = {'toxic': 80,   # >80 not inreae valid auc
                 'severe_toxic': 50,  # can increase lookup from 50
                 'obscene': 80,  # >50 not increase valid auc
                 'threat': 80,   # >40 not increase valid auc
                 'insult': 70,   # can increase lookup from 70
                 'identity_hate': 80} # >30 not increase valid auc
    model = lgb.train(params, train_set=d_train, num_boost_round=rounds_lookup[class_name], valid_sets=watchlist,verbose_eval=10)
    
    print("predict test %s" % class_name)
    submission[class_name] = model.predict(test_df_idf)
    print("predict train %s" % class_name)
    train_preds[class_name] = model.predict(train_tfidf)
    
    del d_train, d_valid, watchlist, model
    gc.collect()
    

Fit toxic
[10]	training's auc: 0.924316	valid_1's auc: 0.91445
[20]	training's auc: 0.950664	valid_1's auc: 0.939425
[30]	training's auc: 0.964818	valid_1's auc: 0.951219
[40]	training's auc: 0.972761	valid_1's auc: 0.958237
[50]	training's auc: 0.977387	valid_1's auc: 0.962321
[60]	training's auc: 0.980612	valid_1's auc: 0.963214
[70]	training's auc: 0.983383	valid_1's auc: 0.965316
[80]	training's auc: 0.985493	valid_1's auc: 0.9661
[90]	training's auc: 0.98727	valid_1's auc: 0.965848
[100]	training's auc: 0.988661	valid_1's auc: 0.965916
[110]	training's auc: 0.989888	valid_1's auc: 0.965769
[120]	training's auc: 0.990915	valid_1's auc: 0.966116
[130]	training's auc: 0.991823	valid_1's auc: 0.966104
[140]	training's auc: 0.992629	valid_1's auc: 0.966358
predict test toxic


/home/luan/.local/lib/python3.5/site-packages/lightgbm/basic.py:447: UserWarning: Converting data to scipy sparse matrix.
  warnings.warn('Converting data to scipy sparse matrix.')


predict train toxic


/home/luan/.local/lib/python3.5/site-packages/ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Fit severe_toxic
[10]	training's auc: 0.965592	valid_1's auc: 0.920757
[20]	training's auc: 0.983912	valid_1's auc: 0.982633
[30]	training's auc: 0.994046	valid_1's auc: 0.983981
[40]	training's auc: 0.996289	valid_1's auc: 0.984186
[50]	training's auc: 0.997551	valid_1's auc: 0.984379
predict test severe_toxic
predict train severe_toxic
Fit obscene
[10]	training's auc: 0.979214	valid_1's auc: 0.979128
[20]	training's auc: 0.987461	valid_1's auc: 0.989109
[30]	training's auc: 0.991848	valid_1's auc: 0.990542
[40]	training's auc: 0.994167	valid_1's auc: 0.991946
[50]	training's auc: 0.995642	valid_1's auc: 0.993556
[60]	training's auc: 0.996645	valid_1's auc: 0.993432
[70]	training's auc: 0.997276	valid_1's auc: 0.99319
[80]	training's auc: 0.997782	valid_1's auc: 0.993031
predict test obscene
predict train obscene
Fit threat
[10]	training's auc: 0.939612	valid_1's auc: 0.914157
[20]	training's auc: 0.975657	valid_1's auc: 0.964327
[30]	training's auc: 0.995004	valid_1's auc: 0.982171
[

In [7]:
train_preds.to_csv(os.path.join(data_folder, 'lgb_train_preds.csv'), index=False)
submission.to_csv(os.path.join(data_folder, 'lgb_submission.csv'), index=False)

In [124]:
def scoring(y_true, y_score, labels):
    score = np.zeros((len(labels),))
    idx = 0
    for label in labels:
        score[idx] = roc_auc_score(y_true[label], y_score[label])
        idx += 1
    score = score.mean()
    return score
        

scoring(train_df, train_preds, class_names)

array([0.98599063, 0.9938711 , 0.99548419, 0.99668764, 0.9893959 ,
       0.9937097 ])

In [28]:
clf = RandomForestClassifier()

In [ ]:
clf.pre

[LogisticRegression调参](http://blog.csdn.net/sun_shengyun/article/details/53811483)
1. penalty惩罚项，[l2,l1]
2. solver优化方法, L2正则化可选四种算法{‘newton-cg','lbfgs','liblinear','sag'}, L1正则化只能选’liblinear'

In [3]:
lr_pipeline = ([('vectorization', CountVectorizer()),
               ('classifier', LogisticRegressionCV())])

In [ ]:
lr_scores = cross_val_score(lr_pipeline, X_train, y_train, scoring = 'roc_auc_score')

#### 持久化
　　externals.joblib包提供了dump和load方法来持久化和加载内存数据：
> #持久化数据  
#第一个参数为内存中的对象  
#第二个参数为保存在文件系统中的名称  
#第三个参数为压缩级别，0为不压缩，3为合适的压缩级别  
dump(grid_search, 'grid_search.dmp', compress=3)  
#从文件系统中加载数据到内存中  
grid_search = load('grid_search.dmp')  

In [83]:
# 保存模型 http://blog.csdn.net/u012102306/article/details/52299589
from sklearn.externals.joblib import dump,load
